In [6]:
import torch
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertAdam, BertConfig, convert_tf_checkpoint_to_pytorch
import numpy as np
from tqdm import tqdm
import pandas as pd
file = pd.read_csv('quora.tsv', '\t')

/Users/LYB/anaconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/LYB/anaconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [208]:
def convert_lines(sentence, max_seq_length, tokenizer):
    max_seq_length -=2
    all_tokens = []
    for text in tqdm(sentence):
        tokens = tokenizer.tokenize(text)
        if len(tokens)>max_seq_length:
            tokens = tokens[:max_seq_length]
        padding_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens+["[SEP]"])+[0] * (max_seq_length - len(tokens))
        all_tokens.append(padding_token)
    return np.array(all_tokens)

In [209]:
max_sequence_length = 200
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', cache_dir=None, do_lower_case=True)
#file['question1'] = file['question1'].astype(str) 
question1 = convert_lines(file["question1"], max_sequence_length, tokenizer)
#file['question2'] = file['question2'].astype(str)
question2 = convert_lines(file["question2"], max_sequence_length, tokenizer)
label = np.array(file['is_duplicate']) 

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/LYB/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
















  0%|          | 0/10 [00:00<?, ?it/s]















100%|██████████| 10/10 [00:00<00:00, 3218.71it/s]















  0%|          | 0/10 [00:00<?, ?it/s]















100%|██████████| 10/10 [00:00<00:00, 3012.93it/s]

In [210]:
import torch.utils.data.dataset as Dataset
import torch.utils.data.dataloader as DataLoader
#创建子类
class modelDataset(Dataset.Dataset):
    #初始化，定义数据内容和标签
    def __init__(self, data1, data2, label):
        self.data1 = data1
        self.data2 = data2
        self.label = label
    #返回数据集大小
    def __len__(self):
        return len(self.label)
    #得到数据内容和标签
    def __getitem__(self, index):
        data1 = torch.LongTensor(self.data1[index])
        data2 = torch.LongTensor(self.data2[index])
        label = torch.Tensor([self.label[index]])
        if torch.cuda.is_available():
            data1 = data1.cuda()
            data2 = data2.cuda()
            label = label.cuda()
        return [data1, data2], label 

In [211]:
dataset = modelDataset(question1, question2, label)

In [218]:
class matchingModel(nn.Module):
    def __init__(self, pre_trained_model):
        super(matchingModel, self).__init__()
        
        self.model_query = BertModel.from_pretrained(pre_trained_model)
        self.model_bidword = BertModel.from_pretrained(pre_trained_model)
        self.dropout = nn.Dropout(p = 0.2)
        self.fc1 = nn.Linear(768*4, 256)
        self.fc2 = nn.Linear(256, 1) 
        
    def infer_metrics(self, x1, x2):
        return torch.cat((x1, x2, torch.abs(x1 - x2), x1 * x2), 1)
    
    def forward(self, input_data):
        
        query_vec = self.model_query(input_data[0])
        bidword_vec = self.model_bidword(input_data[1])
#         similarity_scores = self.infer_metrics(query_vec[1], bidword_vec[1])
#         similarity_scores = self.dropout(torch.tanh(self.fc1(similarity_scores)))
#         similarity_scores = self.fc2(similarity_scores)

        # global average pooling
#         avg_pool_query = torch.mean(query_vec[0][0], 1)
#         avg_pool_bidword = torch.mean(bidword_vec[0][0], 1)
#         # global max pooling
#         max_pool_query, _ = torch.max(query_vec[0][0], 1)
#         max_pool_bidword, _ = torch.max(bidword_vec[0][0], 1)
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        output = cos(query_vec[1], bidword_vec[1])
        
        return output 
 

In [223]:
lr=2e-5
batch_size = 2
accumulation_steps=2
save_steps = 1000
checkpoint = None
EPOCHS = 10
num_train_optimization_steps = int(EPOCHS*len(label) / batch_size / accumulation_steps)
optimizer = BertAdam(model.parameters(),
                     lr=lr,  
                     warmup=0.05,   
                     t_total=num_train_optimization_steps)

model = matchingModel('bert-base-uncased')
model.train() 
criterion = nn.BCELoss()
for epoch in tqdm(range(EPOCHS)): 
#     file_name = 'loss_log_' + 'epoch' + str(epoch) + '.txt'
#     file = open(file_name, 'w', encoding='utf-8')
    train_loader = DataLoader.DataLoader(dataset, batch_size= batch_size, shuffle = True) 
    avg_loss = 0
    avg_accuracy = 0
    optimizer.zero_grad()   
    for x_batch, y_batch in train_loader:
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)  
        loss.backward()  
        optimizer.step() 
        optimizer.zero_grad()
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((y_pred>0.5) == (y_batch>0.5)).to(torch.float)).item()/len(train_loader)
#         i += 1
#         file.write('batch' + str(i) + '\t' + 'avg_loss' + '=' + str(avg_loss) + '\t' + 'avg_accuracy' + '=' + str(avg_accuracy) + '\n')
#     file.close()
#     file_path = output_model_file + str(epoch) +'.bin'
#     torch.save(model.state_dict(), file_path)
    print(f'loss:{avg_loss} accuracy:{avg_accuracy}') 

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/LYB/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/LYB/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/r0/q62_cr2d4yb7h48zdx8tdrs00000gp/T/tmpsx01fuko
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytorch_pretr

loss:2.668728470802307 accuracy:0.2






















 20%|██        | 2/10 [00:44<02:58, 22.27s/it]

loss:2.700250768661499 accuracy:0.2






















 30%|███       | 3/10 [01:07<02:36, 22.42s/it]

...





















 50%|█████     | 5/10 [01:52<01:52, 22.57s/it]

loss:2.76489737033844 accuracy:0.2






















 60%|██████    | 6/10 [02:13<01:28, 22.22s/it]

loss:2.625834274291992 accuracy:0.2


KeyboardInterrupt: 

In [57]:
BERT_MODEL_PATH = '../bert_model/wwm_uncased_L-24_H-1024_A-16/'
bert_config = BertConfig(BERT_MODEL_PATH+'bert_config.json')
tokenizer_1 = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None, do_lower_case=True)

In [58]:
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt', 
BERT_MODEL_PATH + 'bert_config.json',
BERT_MODEL_PATH + 'pytorch_model.bin')
model = BertModel.from_pretrained(BERT_MODEL_PATH)

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /Users/LYB/Desktop/work/bert_model/wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [1024]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [1024]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 1024]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 1024]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 1024]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/gamma

Loading TF weight bert/encoder/layer_14/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_14/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_14/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_14/output/dense/kernel with shape [4096, 1024]
Loading TF weight bert/encoder/layer_15/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_15/attention/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_15/attention/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_15/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_15/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_15/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_15/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_15/attention/self/query/k

Loading TF weight bert/encoder/layer_2/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_2/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_2/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_2/output/dense/kernel with shape [4096, 1024]
Loading TF weight bert/encoder/layer_20/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_20/attention/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_20/attention/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_20/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_20/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_20/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_20/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_20/attention/self/query/kerne

Loading TF weight bert/encoder/layer_4/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_4/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_4/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_4/output/dense/kernel with shape [4096, 1024]
Loading TF weight bert/encoder/layer_5/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_5/attention/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_5/attention/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_5/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_5/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_5/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_5/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_5/attention/self/query/kernel with s

Initialize PyTorch weight ['bert', 'encoder', 'layer_13', 'output', 'LayerNorm', 'beta']
Initialize PyTorch weight ['bert', 'encoder', 'layer_13', 'output', 'LayerNorm', 'gamma']
Initialize PyTorch weight ['bert', 'encoder', 'layer_13', 'output', 'dense', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_13', 'output', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_14', 'attention', 'output', 'LayerNorm', 'beta']
Initialize PyTorch weight ['bert', 'encoder', 'layer_14', 'attention', 'output', 'LayerNorm', 'gamma']
Initialize PyTorch weight ['bert', 'encoder', 'layer_14', 'attention', 'output', 'dense', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_14', 'attention', 'output', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_14', 'attention', 'self', 'key', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_14', 'attention', 'self', 'key', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_14'